In [ ]:
# Notebook to add plots for the stability functions

import numpy as np
import scipy.linalg as linalg
import matplotlib.pyplot as plt

from nodepy import rk

from OrderCondition import *

In [ ]:
def plot_stab(stab_function,color,fill=None,xmin=-10,xmax=10,yminmax=10):
    p,q = stab_function

    # Convert coefficients to floats for speed
    if p.coeffs.dtype=='object':
        p = np.poly1d([float(c) for c in p.coeffs])
    if q.coeffs.dtype=='object':
        q = np.poly1d([float(c) for c in q.coeffs])
        
    u = np.linspace(xmin,xmax,200)
    v = np.linspace(-yminmax,yminmax,200)
    
    U,V = np.meshgrid(u,v)
    Q = U+V*1j
    R=np.abs(p(Q)/q(Q))

    if fill:
        plt.pcolormesh(U, V, np.log(R),cmap=plt.get_cmap('seismic'),vmin=-4, vmax=4)
        plt.colorbar()
    plt.contour(U,V,R,[1],colors=[color],alpha=1,linewidths=3)
    plt.grid()
    
def calc_stab(rkm,bs):
    stab_functions = []
    for b in bs:
        display(b)
        rkm.b = b
        stab_functions.append(rkm.stability_function())
    return stab_functions

def color(chi,color1=[0,0,1],color2=[0,0,0.5]):
    return (chi*color1[0]+(1-chi)*color2[0],chi*color1[1]+(1-chi)*color2[1],chi*color1[2]+(1-chi)*color2[2])

# Implicit case

We are plotting a convex combination of original ex3 method and the embedded backward euler

In [ ]:
ex3 = rk.extrap(3,'implicit euler')

chis = np.linspace(0,1,5)
bs = []
for chi in chis:
    bs.append(chi*ex3.b+(1-chi)*np.array([0,0,0,1/3,1/3,1/3]))

stab_functions = calc_stab(ex3,bs)



In [ ]:
for i in range(5):
    plot_stab(stab_functions[i],color(chis[i]))

plt.savefig('stab_ex3.pdf')

# Explicit case

In [ ]:
ck5_or = rk.loadRKM('CK5').__num__()
ck5 = rk.RungeKuttaMethod(A=ck5_or.A,b=ck5_or.b) #get rid of embedded method

(O,rhs)=OrderCond(ck5.A,ck5.c,order=4)
b_1 =linalg.null_space(O)[:,0]

chis = np.linspace(0,1,5)
alphas = np.linspace(-0.2,0.2,5)
bs = []
for alpha in alphas:
    bs.append(ck5.b+alpha*b_1)

stab_functions = calc_stab(ck5,bs)




In [ ]:
for i in range(5):
    print(i)
    plot_stab(stab_functions[i],color(chis[i]),xmax=5,xmin=-5,yminmax=5)

plt.savefig('stab_ck5.pdf')